# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 22 2023 3:11PM,263433,20,9505209,"Exact-Rx, Inc.",Released
1,May 22 2023 2:45PM,263432,18,EMERSA-9505201,"Emersa Waterbox, LLC",Released
2,May 22 2023 2:40PM,263430,18,EMERSA-9505193,"Emersa Waterbox, LLC",Released
3,May 22 2023 2:39PM,263431,10,Snap-9505191,"Snap Medical Industries, LLC",Released
4,May 22 2023 2:37PM,263429,18,EMERSA-9505190,"Emersa Waterbox, LLC",Released
5,May 22 2023 2:36PM,263428,10,COR00004,"Coronado Aesthetics, LLC",Released
6,May 22 2023 2:34PM,263426,18,EMERSA-9505188,"Emersa Waterbox, LLC",Released
7,May 22 2023 2:34PM,263427,10,Enova-11858,Emerginnova,Released
8,May 22 2023 2:33PM,263425,10,MSP222038,"Methapharm, Inc.",Released
9,May 22 2023 2:33PM,263425,10,MSP222039,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,263429,Released,1
37,263430,Released,1
38,263431,Released,1
39,263432,Released,1
40,263433,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263429,NaN,NaN,1.0
263430,NaN,NaN,1.0
263431,NaN,NaN,1.0
263432,NaN,NaN,1.0
263433,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0.0,67.0,0.0
263369,0.0,1.0,0.0
263374,43.0,29.0,12.0
263376,0.0,0.0,1.0
263377,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0,67,0
263369,0,1,0
263374,43,29,12
263376,0,0,1
263377,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,0,67,0
1,263369,0,1,0
2,263374,43,29,12
3,263376,0,0,1
4,263377,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,,67,
1,263369,,1,
2,263374,43,29,12
3,263376,,,1
4,263377,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc."
1,May 22 2023 2:45PM,263432,18,"Emersa Waterbox, LLC"
2,May 22 2023 2:40PM,263430,18,"Emersa Waterbox, LLC"
3,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC"
4,May 22 2023 2:37PM,263429,18,"Emersa Waterbox, LLC"
5,May 22 2023 2:36PM,263428,10,"Coronado Aesthetics, LLC"
6,May 22 2023 2:34PM,263426,18,"Emersa Waterbox, LLC"
7,May 22 2023 2:34PM,263427,10,Emerginnova
8,May 22 2023 2:33PM,263425,10,"Methapharm, Inc."
11,May 22 2023 2:29PM,263424,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",,,1
1,May 22 2023 2:45PM,263432,18,"Emersa Waterbox, LLC",,,1
2,May 22 2023 2:40PM,263430,18,"Emersa Waterbox, LLC",,,1
3,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",,,1
4,May 22 2023 2:37PM,263429,18,"Emersa Waterbox, LLC",,,1
5,May 22 2023 2:36PM,263428,10,"Coronado Aesthetics, LLC",,,1
6,May 22 2023 2:34PM,263426,18,"Emersa Waterbox, LLC",,,1
7,May 22 2023 2:34PM,263427,10,Emerginnova,,,1
8,May 22 2023 2:33PM,263425,10,"Methapharm, Inc.",,,3
9,May 22 2023 2:29PM,263424,10,Methapharm-G,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",1,,
1,May 22 2023 2:45PM,263432,18,"Emersa Waterbox, LLC",1,,
2,May 22 2023 2:40PM,263430,18,"Emersa Waterbox, LLC",1,,
3,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",1,,
4,May 22 2023 2:37PM,263429,18,"Emersa Waterbox, LLC",1,,
5,May 22 2023 2:36PM,263428,10,"Coronado Aesthetics, LLC",1,,
6,May 22 2023 2:34PM,263426,18,"Emersa Waterbox, LLC",1,,
7,May 22 2023 2:34PM,263427,10,Emerginnova,1,,
8,May 22 2023 2:33PM,263425,10,"Methapharm, Inc.",3,,
9,May 22 2023 2:29PM,263424,10,Methapharm-G,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",1,,
1,May 22 2023 2:45PM,263432,18,"Emersa Waterbox, LLC",1,,
2,May 22 2023 2:40PM,263430,18,"Emersa Waterbox, LLC",1,,
3,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",1,,
4,May 22 2023 2:37PM,263429,18,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",1.0,NaN,NaN
1,May 22 2023 2:45PM,263432,18,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,May 22 2023 2:40PM,263430,18,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
4,May 22 2023 2:37PM,263429,18,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",1.0,0.0,0.0
1,May 22 2023 2:45PM,263432,18,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,May 22 2023 2:40PM,263430,18,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
4,May 22 2023 2:37PM,263429,18,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4214550,36.0,87.0,0.0
15,1580330,76.0,29.0,43.0
18,1053717,4.0,0.0,0.0
19,1053611,11.0,32.0,4.0
20,263433,1.0,0.0,0.0
21,263376,1.0,0.0,0.0
22,263369,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4214550,36.0,87.0,0.0
1,15,1580330,76.0,29.0,43.0
2,18,1053717,4.0,0.0,0.0
3,19,1053611,11.0,32.0,4.0
4,20,263433,1.0,0.0,0.0
5,21,263376,1.0,0.0,0.0
6,22,263369,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,36.0,87.0,0.0
1,15,76.0,29.0,43.0
2,18,4.0,0.0,0.0
3,19,11.0,32.0,4.0
4,20,1.0,0.0,0.0
5,21,1.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,36.0
1,15,Released,76.0
2,18,Released,4.0
3,19,Released,11.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,21,22
Status,,,,,,,
Completed,0.0,43.0,0.0,4.0,0.0,0.0,0.0
Executing,87.0,29.0,0.0,32.0,0.0,0.0,1.0
Released,36.0,76.0,4.0,11.0,1.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,21,22
0,Completed,0.0,43.0,0.0,4.0,0.0,0.0,0.0
1,Executing,87.0,29.0,0.0,32.0,0.0,0.0,1.0
2,Released,36.0,76.0,4.0,11.0,1.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,21,22
0,Completed,0.0,43.0,0.0,4.0,0.0,0.0,0.0
1,Executing,87.0,29.0,0.0,32.0,0.0,0.0,1.0
2,Released,36.0,76.0,4.0,11.0,1.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()